In [ ]:
import dicom2nifti
from glob import glob
import nibabel as nib
import numpy as np
import os
import shutil
import SimpleITK as sitk
from tqdm import tqdm

In [ ]:
# this function separates nifti documents into smaller documents of a defined number of slices

def resample_and_split_nifti(input_nifti, output_dir, num_slices_per_group):
    # Read the NIfTI image
    image = sitk.ReadImage(input_nifti)

    # Get the original size of the image
    original_size = image.GetSize()

    # Calculate the number of groups
    num_groups = original_size[2] // num_slices_per_group

    for group_idx in tqdm(range(num_groups), desc=f"Processing {os.path.basename(input_nifti)}"):
        # Calculate the start and end slice indices for each group
        start_slice = group_idx * num_slices_per_group
        end_slice = (group_idx + 1) * num_slices_per_group

        # Use the Extract method to extract the group
        group = image[:, :, start_slice:end_slice]

        # Get the original file name without extension
        file_name = os.path.splitext(os.path.basename(input_nifti))[0]

        # Save the group as a separate NIfTI file with the group index
        group_output_path = os.path.join(output_dir, f"{file_name}_group_{group_idx}.nii.gz")
        sitk.WriteImage(group, group_output_path)

if __name__ == "__main__":
    input_folder = ""    # Replace with the path to the folder containing NIfTI files
    output_folder = ""  # Replace with the path to the output folder
    num_slices_per_group = 64

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get a list of all NIfTI files in the input folder
    nii_files = [file for file in os.listdir(input_folder) if file.endswith(".nii.gz")]

    for nii_file in nii_files:
        nii_file_path = os.path.join(input_folder, nii_file)
        resample_and_split_nifti(nii_file_path, output_folder, num_slices_per_group)


In [ ]:
def find_empty(in_dir):
    '''
    This function will help you to find the empty volumes that you may not need for your training
    so instead of opening all the files and search for the empty ones, them use this function to make it quick.
    '''
    
    list_patients = []
    for patient in sorted(glob(os.path.join(in_dir, '*'))):
        img = nib.load(patient)

        if len(np.unique(img.get_fdata())) == 1:
            print(os.path.basename(os.path.normpath(patient)))
            list_patients.append(os.path.basename(os.path.normpath(patient)))
    
    return list_patients

In [ ]:
#define the directory and run find_empty 
in_dir = ''
find_empty(in_dir)

In [ ]:
# this function takes in a list and deletes everything. Use the printed list_patients as input.
def delete_files_in_directory(directory_path, file_list):
    try:
        # Check if the directory exists
        if not os.path.exists(directory_path):
            print(f"Error: Directory '{directory_path}' does not exist.")
            return

        # Loop through the file list and delete each file
        for file_name in file_list:
            file_path = os.path.join(directory_path, file_name)

            # Check if the file exists before attempting to delete
            if os.path.exists(file_path):
                os.remove(file_path)
                print(f"File '{file_name}' deleted successfully.")
            else:
                print(f"Warning: File '{file_name}' not found in '{directory_path}'.")

    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    directory_path = ""  # Replace with the path to the directory containing the files to delete
    files_to_delete = [] # Replace with the list of files to be deleted

    delete_files_in_directory(directory_path, files_to_delete)

In [ ]:
# This function allows you to extract a list of files that have less than a defined percentage of
# slices with pancreas. Percentage chosen = 30%.
def find_slices_with_ones(nii_file_path):
    # Load the NIfTI file
    img = nib.load(nii_file_path)

    # Get the data array from the NIfTI file
    data = img.get_fdata()

    # Find the slices containing 1s
    slices_with_ones = []
    for slice_idx in range(data.shape[2]):
        slice_data = data[:, :, slice_idx]
        if np.any(slice_data == 1):
            slices_with_ones.append(slice_idx)

    # Calculate the percentage of slices with ones
    percentage_with_ones = (len(slices_with_ones) / data.shape[2]) * 100

    return percentage_with_ones, slices_with_ones

if __name__ == "__main__":
    folder_path = '/home/alialhajabdo/Documents/Subtask1/TrainMask64/'
    files_with_low_ones_percentage = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.nii.gz'):
            nii_file_path = os.path.join(folder_path, filename)
            percentage, slices_with_ones = find_slices_with_ones(nii_file_path)
            
            if percentage < 30:
                files_with_low_ones_percentage.append(filename)

            print(f"{filename}: {percentage:.2f}% of slices with ones")

    if len(files_with_low_ones_percentage) > 0:
        print("Files with less than 30% slices with ones:")
        for filename in files_with_low_ones_percentage:
            print(filename)
    else:
        print("All files have at least 30% slices with ones.")
